[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [4]:
from ecdsa import SigningKey, SECP256k1

private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key()

message = b"I need coffee"
signature = private_key.sign(message)

In [5]:
public_key.verify(signature, message)

True

In [6]:
# if we use a different message, we get an error such as belong which
# which can be caught and handled by another part of our program
public_key.verify(signature, b"I don't need coffee")

BadSignatureError: 

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [7]:
class Transfer:
    
    def __init__(self, signature, public_key):
        self.signature = signature
        self.public_key = public_key
        
class ECDSACoin:
    
    def __init__(self, transfers):
        self.transfers = transfers

In [8]:
# Generate private and public keys for the usual characters 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [35]:
from utils import serialize

def issue(public_key):
    # get a private key and derive public key 
    # and serialize it into a message
    message = serialize(public_key)
    
    # create a signature (signing the message), bytestring
    signature = bank_private_key.sign(message)    
    
    # create a transfer which references that signature and public key
    transfer = Transfer(
        signature=signature,
        public_key=public_key
    )
    
    # use that transfer as the only transfer in a list of transfers for
    # this coin
    coin = ECDSACoin([transfer])
    # and use that to return to us the bank-issued coin
    return coin

# Validating the First Transfer

In [33]:
def validate(coin):
    # take the first transfer item of the coin to check it cam from bank
    transfer = coin.transfers[0]
    
    # serialize the transfer public key (recipient's pub key) to reveal
    # message which should be signed
    message = serialize(transfer.public_key)
    
    # check that the bank signed the message
    bank_public_key.verify(transfer.signature, message)

In [34]:
# issue one coin to alice by passing her public key as the message to issue
alice_coin = issue(alice_public_key)

# validate that the coin belongs to alice
validate(alice_coin)

In [12]:
# now we will try to make a "fake" coin to check that we will detect fakes
# bob is going to try and issue a new coin to himself
message = serialize(bob_public_key)
    
# sign using bob's key (not authorized to issue new coins)
signature = bob_private_key.sign(message)    

# create a transfer which references that signature and public key
transfer = Transfer(
    signature=signature,
    public_key=bob_public_key
)

# make (issue) a new coin which only has this single transfer from bob
# to himself
alt_coin = ECDSACoin([transfer])

In [13]:
# here we will use python to catch any errors
from ecdsa import BadSignatureError

# use 'try' when you attempt to run some code which might fail
try:
    validate(alt_coin)

# if the below exception is raised, do something (print the message below)
except BadSignatureError:
    print("Bad signature detected")
    

Bad signature detected


# Validating Subsequent Transfers

In [14]:
# for subsequent tx we will combine the prev. sig with next sig in the
# new messge (next pub key)

def transfer_message(previous_signature, next_owner_public_key):
    # return a serialized dictionary of signatures
    return serialize({
        "previous_signature": previous_signature,
        "next_owner_public_key": next_owner_public_key,
    })

def validate(coin):
    # check the first transfer
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)
    
    # check the rest of the coin.transfers
    # keep a pointer to previous transfer and use that pub key to verify subsequent signature
    previous_transfer = coin.transfers[0]
    
    # [1:] means "start at this index, and go to the end"
    for next_transfer in coin.transfers[1:]:
        
        # new message will be previous signature and the pub key of the intended recipient
        message = transfer_message(previous_transfer.signature, next_transfer.public_key)
        previous_transfer.public_key.verify(
            next_transfer.signature,
            message,
        )       
    

In [22]:
# given a key, look up in database who is the owner
def get_owner(coin):
    database = {
        # make a map of keys to names
        bob_public_key: "Bob",
        alice_public_key: "Alice",
        bank_public_key: "Bank",
    }
    public_key = coin.transfers[-1].public_key
    # return the owner of the key (coin)
    return database[public_key]

In [39]:
# test
# issue alice a new coin
coin = issue(alice_public_key)

# print who the current owner of the coin is
print("This coin is owned by ", get_owner(coin))

# create a new signed message passing the coin to bob('s public key)
# use -1 here to always check the last signature in the chain
message = transfer_message(coin.transfers[-1].signature, bob_public_key)

# initiate the transfer to bob
alice_to_bob = Transfer(
    # sender private key signing
    signature = alice_private_key.sign(message),
    # recipient pub key
    public_key = bob_public_key
)

# make the transfer and append to transfers
coin.transfers.append(alice_to_bob)

# print who the new owner is
print("This coin is owned by ", get_owner(coin))


# now to test it really works, we will have bob send it back to the bank...
message = transfer_message(coin.transfers[-1].signature, bank_public_key)

bob_to_bank = Transfer(
    signature = bob_private_key.sign(message),
    public_key = bank_public_key
)

coin.transfers.append(bob_to_bank)

print("This coin is owned by ", get_owner(coin))

This coin is owned by  Alice
This coin is owned by  Bob
This coin is owned by  Bank


# Serialization

In [41]:
from utils import to_disk, from_disk

In [42]:
# write the coin to disk, and then read it back
import os

# before we write check to see if the file is already there
filename = "coin.ecdsacoin"

# this os.path.isfile is platform agnostic
print("Does the coinfile exist on disk?", os.path.isfile(filename))

Does the coinfile exist on disk? False


In [43]:
coin = issue(alice_public_key)

to_disk(coin, filename)

In [44]:
# check that the file is there afterwards too
print("Does the coinfile exist on disk?", os.path.isfile(filename))

Does the coinfile exist on disk? True


In [46]:
coin = from_disk(filename)
coin

# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [21]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

alice_to_bob = Transfer(
    signature=alice_private_key.sign(transfer_message(coin.transfers[-1].signature, bob_public_key)),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()